## **ドライブをマウント**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

**マウントされているかチェック**

In [0]:
!ls drive

### **地形生成**

In [0]:
# *** パッケージのインポート***
import numpy as np
import itertools
import os 
import matplotlib.pyplot as plt
import random


# 変数設定 -----------------------------------------
N = 12 #要素の数
i = 1000 #繰り返しの数

file_name = os.path.expanduser("~") # 保存先の設定（初期設定）

# *** 地形生成関数の定義 ***************************************

def imatrix_rand(D,K):
    '''
    適応度地形の生成。
    地形の要素同士に関係性が存在すると仮定（関係あり：1、　関係なし：0）
    '''
    P = N/D
    
    Int_matrix_rand = np.zeros((N, N))
    for aa1 in np.arange(0,N,P,dtype = int):
      Ln = aa1 + P
      for aa2 in np.arange(aa1,Ln,1,dtype = int):
        Indexes_1 = list(range(N))
        for i in np.arange(aa1,Ln,1,dtype = int):
          Indexes_1.remove(i)  
          np.random.shuffle(Indexes_1)
        for i in np.arange(aa1,Ln,1,dtype = int):
          Indexes_1.append(i)
          Chosen_ones = Indexes_1[-(K+1):]  
        for aa3 in Chosen_ones:
          Int_matrix_rand[aa2, aa3] = 1  
    return(Int_matrix_rand)

# *** 貢献度計算の関数定義 ***********************************************
def calc_fit(NK_land_, inter_m, Current_position, Power_key_):
    '''
    要素ごとの貢献度（0.0〜1.0の間）。
    組み合わせに対して要素ごとの適応地形を掛け合わせて算出。
    '''
    Fit_vector = np.zeros(N)
    for ad1 in np.arange(N):
        Fit_vector[ad1] = NK_land_[np.sum(Current_position * inter_m[ad1]
                                          * Power_key_), ad1]
    return(Fit_vector)


def comb_and_values(NK_land_, Power_key_, inter_m):
    '''
    組み合わせごとの貢献度表。
    calc＿fit関数を用いて全組み合わせの貢献度を算出。
    ローカルピーク（極大地点）とピーク（最大地点）も算出。
    '''
    Comb_and_value = np.zeros((2**N, N*2+3))  # 記録用の配列
    c1 = 0  
    for c2 in itertools.product(range(2), repeat=N):
        Combination1 = np.array(c2)  
        fit_1 = calc_fit(NK_land_, inter_m, Combination1, Power_key_)
        Comb_and_value[c1, :N] = Combination1  
        Comb_and_value[c1, N:2*N] = fit_1
        Comb_and_value[c1, 2*N] = np.mean(fit_1)
        c1 = c1 + 1
    for c3 in np.arange(2**N): # 隣接している状態のどれよりも高い貢献度を持っている　→　ローカルピークである 
        loc_p = 1  
        for c4 in np.arange(N): 
            new_comb = Comb_and_value[c3, :N].copy().astype(int)
            new_comb[c4] = abs(new_comb[c4] - 1)
            if ((Comb_and_value[c3, 2*N] <
                 Comb_and_value[np.sum(new_comb*Power_key_), 2*N])):
                loc_p = 0  
        Comb_and_value[c3, 2*N+1] = loc_p
    max_ind = np.argmax(Comb_and_value[:, 2*N])
    Comb_and_value[max_ind, 2*N+2] = 1
    return(Comb_and_value)

Power_key = np.power(2, np.arange(N - 1, -1, -1))  # 組み合わせを2進数→10進数に変換する際に利用。
Landscape_data = np.zeros((i, 2**N, N*2+3))  # 貢献度表を記録しておく配列

# K,Dの値を先に準備 
KD_matrix = \
  np.array([
            [2,5,6,7],
            [3,3,4,5],
            [4,2,3,4],
            [6,1,2,3]
            ])

for DD in [0,1,2,3]:
  for KK in [1,2,3]:
    D = KD_matrix[DD,0]
    K = KD_matrix[DD,KK]
    for i_1 in np.arange(i):
      Int_matrix = imatrix_rand(D, K).astype(int)
      NK_land = np.random.rand(2**N, N)  
      Landscape_data[i_1] = comb_and_values(NK_land, Power_key, Int_matrix)

# ***　結果のまとめ ****************************************
    number_of_peaks = np.zeros(i)
    max_values = np.zeros(i)
    min_values = np.zeros(i)
    for i_2 in np.arange(i):
      number_of_peaks[i_2] = np.sum(Landscape_data[i_2, :, 2*N+1])
      max_values[i_2] = np.max(Landscape_data[i_2, :, 2*N])
      min_values[i_2] = np.min(Landscape_data[i_2, :, 2*N])

# *** 結果の出力＆保存 ************
    print('結果の表示ーーー N = ' + str(N) + ', D = ' + str(D) + ', K = ' + str(K) + ' ---')
    print('地形の山の数（平均）: ' + str(np.mean(number_of_peaks)))
    print('地形の山の数（最大）: ' + str(np.max(number_of_peaks)))
    print('地形の山の数（最小）: ' + str(np.min(number_of_peaks)))
    print('貢献度の最大値（平均）: ' + str(np.mean(max_values)))
    print('貢献度の最小値（平均）: ' + str(np.mean(min_values)))
    print('-' * 16)

# 地形の極大地点数（ヒストグラム）
    '''
    ヒストグラムを表示する

    plt.figure(1, facecolor='white', figsize=(8, 6), dpi=150)  
    plt.hist(number_of_peaks, bins=10, range=(1, 20), color='dodgerblue', edgecolor='black') # adjust if necessary
    plt.title('Distribution of the number of peaks', size=12)
    plt.xlabel('number of peaks', size=10)
    plt.ylabel('frequency', size=10)

    上下の３つのクオーテーションを消して実行すると、大量にプロットが表示されて大変なので注意！！
    '''


    if not os.path.exists(file_name + '/NK_workshop/'):
      os.makedirs(file_name + '/NK_workshop/')
    np.save(file_name + '/NK_workshop/NK_land_N_' + str(N) + '_D_' + str(D) +
        '_K_' + str(K) + '_i_' + str(i) + '.npy', Landscape_data)

print('*' * 16 + '実行完了' + '*' * 16)

# 終わり

### **実行＆探索**

In [0]:
# *** パッケージのインポート***
import numpy as np
from os.path import expanduser 
import matplotlib.pyplot as plt
import csv
from google.colab import drive
import os

# 変数設定　＊＊Nは上と揃えること＊＊ -----------------------------------------
N = 12  # 要素数  
i = 1000  # 期数
t = 25   #一期あたりの回数
reorg = 100 
file_name = expanduser("~") # 上で保存した場所を指定する   

# 変数の初期化　-----------------------------------------
D_original = 1
count_cycle = 0
fitness3 = [0]

# *** 必要な関数を定義****************************************************                                             
def select_divisor (N):
  '''
  指定された数字の約数を配列で返す
  '''
  selected_divisor = [0]
  for divisor in np.arange( 2, N+1, 1, dtype = int):
    if( N % divisor == 0):
      selected_divisor.append( divisor )
  return selected_divisor

def D_prob_make (D_array):
  '''
  Dの選択確率を初期化する
  '''
  array = [0]
  for i0 in np.arange(0, len(D_array), 1, dtype = int):
    array.append(1/len(D_array))
  del array[0]
  return array

def norm_rec_make(i, D_array):
  '''
  正規化された貢献度を記録しておく配列の作成
  '''
  array = np.zeros((i+1, len(D_array)))
  for ii in np.arange(0,len(D_array), 1, dtype = int):
    array[0,ii] = 1
  return array

# *** 実行（D、Kの全パターンを配列として準備してから実行） *****************************************

KD_matrix = \
  np.array([
            [2,5,6,7],
            [3,3,4,5],
            [4,2,3,4],
            [6,1,2,3]
            ])

D_array_original = select_divisor(N) #約数を格納するリスト
Landscape_data = np.zeros((12,len(D_array_original)+6))

for DD in [0,1,2,3]:
  for KK in [1,2,3]:
    D = KD_matrix[DD,0]
    K = KD_matrix[DD,KK]
    D_original = D
    P_original = int(N/D)
    # *** ファイルの読み込み **************
    NK_landscape = np.load(file_name + '/NK_workshop/NK_land_N_' + str(N) + '_D_' + str(D) +
                          '_K_' + str(K) + '_i_' + str(i) + '.npy')
    # **********************************

    power_key = np.power(2, np.arange(N - 1, -1, -1))  # 2進数を10進数へ直すための配列
    D_prob_array = [float(0)] #np.random.choiceの加重を決定するリスト
    D_norm_rec = norm_rec_make(i, D_array_original) #正規化された貢献度の値の記録を格納する配列
    D_prob_rec = np.zeros((i,len(D_array_original))) #Dの選択確率をそれぞれ記録しておく配列
    norm_ave = D_prob_make(D_array_original) #normの平均（あるいはその調整値）を格納する配列
    learn_strength = 23 #学習の強度
    final_max_fitness = 0 #fitness_normの最高を記録していく

    D_prob_array = D_prob_make(D_array_original)

    Output3 = np.zeros((i, t))
    for i1 in np.arange(i):
      combination = np.random.binomial(1, 0.5, N)  # 最初の組み合わせ設定
      row = np.sum(combination*power_key)
      fitness = NK_landscape[i1, row, 2*N]
      max_fit = np.max(NK_landscape[i1, :, 2*N])  # 貢献度の最大値
      min_fit = np.min(NK_landscape[i1, :, 2*N])  # 貢献度の最小値
      fitness_norm = (fitness - min_fit)/(max_fit - min_fit) # 貢献度は正規化をして他と比較
      if(0.7 < np.random.rand()):
        D= np.random.choice(D_array_original)
        #prob_arrayの確率に従ってランダム選択
      else:
        D= np.random.choice(D_array_original, p = D_prob_array)     
      for t1 in np.arange(t):
        Output3[i1, t1] = fitness_norm
        row = np.sum(combination*power_key)
        if D != 0:  # 以下、分権的なローカルサーチ（探索）を行う
          P = int(N/D) # 部門ごとの意思決定数
          comb_D = np.hsplit(combination, D) # 組み合わせを部門ごとに分ける
          new_fits = np.zeros(P+1) 
          new_fits[P] = fitness 
          final_combs_D = np.zeros(N)
          for i2 in np.arange(D): 
            s = int(i2*P)
            new_comb_D = comb_D[i2].copy() # 部門ごとの組み合わせを取得
            new_combs = np.zeros((D,P+1,N))
            new_combs[i2,P,:] = combination.copy()
            # 以下、変更後の判定
            for i3 in np.arange(P):
              new_comb_D[i3]= abs(new_comb_D[i3] - 1) # 0→1、1→0へ
              new_combs[i2,i3,:] = combination.copy()
              new_combs[i2,i3,s:s+P] = new_comb_D
              row2 = int(np.sum(new_combs[i2,i3,:]*power_key))
              new_fits[i3] = NK_landscape[i1, row2, 2*N]  # 変更先の貢献度の平均
              new_comb_D = comb_D[i2].copy()
            final_combs_D[s:s+P] = new_combs[i2,np.argmax(new_fits),s:s+P]
            combination = final_combs_D
            row = int(np.sum(combination*power_key))
            new_fitness = NK_landscape[i1, row, 2*N]  # 最終的な貢献 
            if new_fitness > fitness:
              fitness = new_fitness.copy()
            elif D == 0:  # 集権的なローカルサーチ（探索）
              new_combination = combination.copy()
              choice_var = np.random.randint(N)
              new_combination[choice_var] = abs(new_combination[choice_var] - 1)
              row = np.sum(new_combination*power_key)
              new_fitness = NK_landscape[i1, row, 2*N]
              if new_fitness > fitness:
                combination = new_combination.copy()
                fitness = new_fitness.copy()
      fitness_norm = (fitness - min_fit)/(max_fit - min_fit) # 新たに貢献度を正規化
      D_norm_rec[i1+1, D_array_original.index(D)] = fitness_norm #正規化した貢献度を記録する
      
      # 過去の正規化した貢献度の平均を計算し、累乗することで調整をかける
      for i2 in np.arange(0, len(D_array_original), 1, dtype = int):
        norm_ave[i2] = (np.sum(D_norm_rec[0:, i2])  / np.count_nonzero(D_norm_rec[0:, i2])) ** learn_strength
      # 調整後の正規化した貢献度の値をもとに、Dの選択確率を求める
      for i3 in np.arange(0, len(D_array_original), 1, dtype = int):
        D_prob_array[i3] = norm_ave[i3] / np.sum(norm_ave)
        D_prob_rec[i1,i3] = D_prob_array[i3]
      # 正規化した貢献度の最大値算出用
      if final_max_fitness < fitness_norm:
          final_max_fitness = fitness_norm


# *** 結果の表示　***************************************************************

    ave_norm = np.sum(D_norm_rec) / 1000
    Landscape_data[count_cycle,0] = D_original
    Landscape_data[count_cycle,1] = K
    Landscape_data[count_cycle,2] = K-P_original+1
    Landscape_data[count_cycle,3] = final_max_fitness
    Landscape_data[count_cycle,4] = ave_norm
    Landscape_data[count_cycle,5] = D_array_original[np.argmax(D_prob_array)]
    count_figure = 6
    for iii3 in range(len(D_array_original)):
      Landscape_data[count_cycle, count_figure] = D_prob_array[iii3]
      count_figure += 1

    count_cycle += 1

    print("LandscapeのD:" + str(D_original))
    print("Kの値：" + str(K))
    print("最大のNorm:"+str(final_max_fitness))
    print("normの平均："+str(ave_norm))
    print('--Dの選択確率------')
    for iii2 in range(len(D_array_original)):
      print("D=" + str(D_array_original[iii2])+ "の選択確率：" + str(D_prob_array[iii2])) 
    print("Dの選択確率が最大の際のＤ："+ str(D_array_original[np.argmax(D_prob_array)]))
    print()

    plt.figure(facecolor='white', figsize=(8, 6), dpi=150)  
    for iii4 in range(len(D_array_original)):
      if iii4 == 0:
        plt.plot(D_prob_rec[0:,iii4], label = 'cen')
      else:
       plt.plot(D_prob_rec[0:,iii4], label = str(D_array_original[iii4]))
    plt.ylim(0, 0.7)
    plt.xlim(0,1000)
    plt.legend(loc=4,prop={'size':10})
    plt.title('Trasition of selection probability of Mt (G='+str(D_original) +', K='+ str(K)+ ', Kex=' +str(K-P_original+1)+')', size=12)
    plt.xlabel('Time Periods', size=12)
    plt.ylabel('Selection Probability of Mt', size=12)
    plt.legend(loc='upper left', bbox_to_anchor=(1,1))
    plt.savefig(file_name + '/NK_workshop/_D_' + str(D_original) +
                          '_K_' + str(K) + '_i_' + str(i)+'10.jpg', format='jpg')
    if not os.path.exists(file_name + '/NK_workshop/'):
      os.makedirs(file_name + '/NK_workshop/')
    np.savetxt(file_name + '/NK_workshop/_D_'+str(D)+'_Kex_'+str(K-P_original+1)+'_norm_rec_10.csv',D_norm_rec,delimiter=",")
    print(file_name + '/NK_workshop/result_for_figure10')
    print("--*****************************************-----")



if not os.path.exists(file_name + '/NK_workshop/'):
    os.makedirs(file_name + '/NK_workshop/')
np.savetxt(file_name + '/NK_workshop/result_for_figur10.csv',Landscape_data,delimiter=",")
print(file_name + '/NK_workshop/result_for_figure')

      #　終わり
